# Python Backtest Comparison - Quant Explorer Equivalent

This notebook provides a Python implementation equivalent to the Elixir `backtest_examples.livemd` for direct comparison.

## Features
- **Multi-Strategy Backtesting**: SMA, EMA, RSI, MACD strategies
- **Interactive Visualizations**: Plotly charts with buy/sell signals
- **Performance Metrics**: Returns, drawdowns, Sharpe ratios, win rates
- **Benchmark Comparison**: Against buy & hold strategy
- **Real Market Data**: Using yfinance for AAPL and MSFT

## Setup and Installation

In [3]:
# Install required packages (run once)
# !pip install yfinance pandas numpy matplotlib plotly scipy nbformat

In [4]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("✓ All packages imported successfully")
print(f"📊 Pandas version: {pd.__version__}")
print(f"🔢 NumPy version: {np.__version__}")

✓ All packages imported successfully
📊 Pandas version: 2.3.3
🔢 NumPy version: 2.3.3


## Strategy Base Class

This mirrors the behavior-driven design from the Elixir implementation.

In [5]:
class TradingStrategy:
    """Base class for trading strategies - mirrors Elixir Quant.Strategy behavior"""
    
    def __init__(self, name, initial_capital=100000, commission=0.001, slippage=0.0005):
        self.name = name
        self.initial_capital = initial_capital
        self.commission = commission
        self.slippage = slippage
        
    def generate_signals(self, data):
        """Override in subclasses to generate trading signals"""
        raise NotImplementedError
        
    def backtest(self, data):
        """Run backtest for the strategy"""
        df = data.copy()
        
        # Generate signals
        signals = self.generate_signals(df)
        df['signal'] = signals
        
        # Initialize portfolio tracking
        df['position'] = 0.0
        df['portfolio_value'] = self.initial_capital
        df['cash'] = self.initial_capital
        df['holdings'] = 0.0
        df['trades'] = 0
        
        position = 0.0
        cash = self.initial_capital
        trades = []
        
        for i in range(1, len(df)):
            signal = df.iloc[i]['signal']
            price = df.iloc[i]['close']
            
            if signal == 1 and position == 0:  # Buy signal
                shares_to_buy = cash / (price * (1 + self.commission + self.slippage))
                position = shares_to_buy
                cash = cash - (shares_to_buy * price * (1 + self.commission + self.slippage))
                trades.append({'date': df.index[i], 'type': 'buy', 'price': price, 'shares': shares_to_buy})
                
            elif signal == -1 and position > 0:  # Sell signal
                cash = cash + (position * price * (1 - self.commission - self.slippage))
                trades.append({'date': df.index[i], 'type': 'sell', 'price': price, 'shares': position})
                position = 0.0
            
            df.iloc[i, df.columns.get_loc('position')] = position
            df.iloc[i, df.columns.get_loc('cash')] = cash
            df.iloc[i, df.columns.get_loc('holdings')] = position * price
            df.iloc[i, df.columns.get_loc('portfolio_value')] = cash + (position * price)
            df.iloc[i, df.columns.get_loc('trades')] = len(trades)
        
        # Calculate performance metrics
        returns = df['portfolio_value'].pct_change().dropna()
        
        results = {
            'data': df,
            'trades': trades,
            'final_value': df['portfolio_value'].iloc[-1],
            'total_return': (df['portfolio_value'].iloc[-1] - self.initial_capital) / self.initial_capital,
            'max_drawdown': self._calculate_max_drawdown(df['portfolio_value']),
            'win_rate': self._calculate_win_rate(trades),
            'trade_count': len(trades),
            'sharpe_ratio': self._calculate_sharpe_ratio(returns),
            'volatility': returns.std() * np.sqrt(252)
        }
        
        return results
    
    def _calculate_max_drawdown(self, portfolio_values):
        peak = portfolio_values.expanding(min_periods=1).max()
        drawdown = (portfolio_values - peak) / peak
        return drawdown.min()
    
    def _calculate_win_rate(self, trades):
        if len(trades) < 2:
            return 0.0
        winning_trades = 0
        for i in range(1, len(trades), 2):
            if i < len(trades):
                buy_price = trades[i-1]['price']
                sell_price = trades[i]['price']
                if sell_price > buy_price:
                    winning_trades += 1
        total_trade_pairs = len(trades) // 2
        return winning_trades / total_trade_pairs if total_trade_pairs > 0 else 0.0
    
    def _calculate_sharpe_ratio(self, returns, risk_free_rate=0.02):
        if len(returns) == 0 or returns.std() == 0:
            return 0.0
        excess_returns = returns.mean() * 252 - risk_free_rate
        return excess_returns / (returns.std() * np.sqrt(252))

print("✓ TradingStrategy base class defined")

✓ TradingStrategy base class defined


## Strategy Implementations

These mirror the exact same strategies from the Elixir implementation.

In [ ]:
class SMAStrategy(TradingStrategy):
    """Simple Moving Average Crossover Strategy"""
    
    def __init__(self, fast_period=10, slow_period=30, **kwargs):
        super().__init__(f"SMA({fast_period},{slow_period})", **kwargs)
        self.fast_period = fast_period
        self.slow_period = slow_period
    
    def generate_signals(self, data):
        df = data.copy()
        df[f'sma_{self.fast_period}'] = df['close'].rolling(window=self.fast_period).mean()
        df[f'sma_{self.slow_period}'] = df['close'].rolling(window=self.slow_period).mean()
        
        signals = np.zeros(len(df))
        
        for i in range(1, len(df)):
            fast_current = df.iloc[i][f'sma_{self.fast_period}']
            slow_current = df.iloc[i][f'sma_{self.slow_period}']
            fast_prev = df.iloc[i-1][f'sma_{self.fast_period}']
            slow_prev = df.iloc[i-1][f'sma_{self.slow_period}']
            
            if (fast_current > slow_current and fast_prev <= slow_prev and 
                not pd.isna(fast_current) and not pd.isna(slow_current)):
                signals[i] = 1
            elif (fast_current < slow_current and fast_prev >= slow_prev and 
                  not pd.isna(fast_current) and not pd.isna(slow_current)):
                signals[i] = -1
                
        return signals

class EMAStrategy(TradingStrategy):
    """Exponential Moving Average Crossover Strategy"""
    
    def __init__(self, fast_period=12, slow_period=26, **kwargs):
        super().__init__(f"EMA({fast_period},{slow_period})", **kwargs)
        self.fast_period = fast_period
        self.slow_period = slow_period
    
    def generate_signals(self, data):
        df = data.copy()
        df[f'ema_{self.fast_period}'] = df['close'].ewm(span=self.fast_period).mean()
        df[f'ema_{self.slow_period}'] = df['close'].ewm(span=self.slow_period).mean()
        
        signals = np.zeros(len(df))
        
        for i in range(1, len(df)):
            fast_current = df.iloc[i][f'ema_{self.fast_period}']
            slow_current = df.iloc[i][f'ema_{self.slow_period}']
            fast_prev = df.iloc[i-1][f'ema_{self.fast_period}']
            slow_prev = df.iloc[i-1][f'ema_{self.slow_period}']
            
            if (fast_current > slow_current and fast_prev <= slow_prev):
                signals[i] = 1
            elif (fast_current < slow_current and fast_prev >= slow_prev):
                signals[i] = -1
                
        return signals

class RSIStrategy(TradingStrategy):
    """RSI Mean Reversion Strategy - FIXED to match Elixir implementation"""
    
    def __init__(self, period=14, oversold=30, overbought=70, **kwargs):
        super().__init__(f"RSI({period},{oversold},{overbought})", **kwargs)
        self.period = period
        self.oversold = oversold
        self.overbought = overbought
    
    def generate_signals(self, data):
        df = data.copy()
        
        # Calculate RSI
        delta = df['close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=self.period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=self.period).mean()
        rs = gain / loss
        df['rsi'] = 100 - (100 / (1 + rs))
        
        signals = np.zeros(len(df))
        
        # FIXED: Match Elixir logic - immediate signals, not crossover
        for i in range(len(df)):
            rsi = df.iloc[i]['rsi']
            
            # Immediate signal when in threshold territory (like Elixir)
            if pd.notna(rsi):
                if rsi <= self.oversold:  # Oversold -> Buy
                    signals[i] = 1
                elif rsi >= self.overbought:  # Overbought -> Sell
                    signals[i] = -1
                # Neutral zone -> Hold (0)
                
        return signals

class MACDStrategy(TradingStrategy):
    """MACD Crossover Strategy"""
    
    def __init__(self, fast_period=12, slow_period=26, signal_period=9, **kwargs):
        super().__init__(f"MACD({fast_period},{slow_period},{signal_period})", **kwargs)
        self.fast_period = fast_period
        self.slow_period = slow_period
        self.signal_period = signal_period
    
    def generate_signals(self, data):
        df = data.copy()
        
        # Calculate MACD
        ema_fast = df['close'].ewm(span=self.fast_period).mean()
        ema_slow = df['close'].ewm(span=self.slow_period).mean()
        df['macd'] = ema_fast - ema_slow
        df['macd_signal'] = df['macd'].ewm(span=self.signal_period).mean()
        df['macd_histogram'] = df['macd'] - df['macd_signal']
        
        signals = np.zeros(len(df))
        
        for i in range(1, len(df)):
            macd = df.iloc[i]['macd']
            macd_signal = df.iloc[i]['macd_signal']
            prev_macd = df.iloc[i-1]['macd']
            prev_macd_signal = df.iloc[i-1]['macd_signal']
            
            if macd > macd_signal and prev_macd <= prev_macd_signal:
                signals[i] = 1
            elif macd < macd_signal and prev_macd >= prev_macd_signal:
                signals[i] = -1
                
        return signals

print("✓ All strategy classes defined")
print("  - SMAStrategy (Simple Moving Average Crossover)")
print("  - EMAStrategy (Exponential Moving Average Crossover)")
print("  - RSIStrategy (RSI Mean Reversion) - FIXED to match Elixir")
print("  - MACDStrategy (MACD Crossover)")
print("\n🔧 Key Fix: RSI strategy now generates immediate signals when in threshold territory")
print("   (matching Elixir implementation), not crossover signals")

✓ All strategy classes defined
  - SMAStrategy (Simple Moving Average Crossover)
  - EMAStrategy (Exponential Moving Average Crossover)
  - RSIStrategy (RSI Mean Reversion)
  - MACDStrategy (MACD Crossover)


## Data Fetching

Fetch the same symbols used in the Elixir implementation.

In [7]:
def fetch_data(symbol, period="2y"):
    """Fetch stock data using yfinance"""
    try:
        ticker = yf.Ticker(symbol)
        data = ticker.history(period=period)
        data.columns = data.columns.str.lower()
        return data
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None

def calculate_buy_hold_performance(data, initial_capital):
    """Calculate buy and hold benchmark performance"""
    initial_price = data['close'].iloc[0]
    shares = initial_capital / initial_price
    return data['close'] * shares

# Fetch data for AAPL and MSFT
symbols = ['AAPL', 'MSFT']
market_data = {}

for symbol in symbols:
    print(f"📈 Fetching data for {symbol}...")
    data = fetch_data(symbol, period="2y")
    if data is not None:
        market_data[symbol] = data
        print(f"   ✓ Shape: {data.shape}")
        print(f"   ✓ Date range: {data.index[0].date()} to {data.index[-1].date()}")
    else:
        print(f"   ✗ Failed to fetch data for {symbol}")

print(f"\n📊 Successfully fetched data for {len(market_data)} symbols")

📈 Fetching data for AAPL...
   ✓ Shape: (501, 7)
   ✓ Date range: 2023-10-16 to 2025-10-14
📈 Fetching data for MSFT...
   ✓ Shape: (501, 7)
   ✓ Date range: 2023-10-16 to 2025-10-14

📊 Successfully fetched data for 2 symbols


## Strategy Testing - AAPL

Test the same SMA(20,50) strategy on AAPL as in the Elixir implementation.

In [8]:
if 'AAPL' in market_data:
    print("🍎 AAPL Strategy Testing")
    print("=" * 40)
    
    aapl_data = market_data['AAPL']
    
    # Test SMA(20,50) strategy - same as Elixir implementation
    sma_strategy = SMAStrategy(fast_period=20, slow_period=50)
    
    print(f"Testing {sma_strategy.name} on AAPL...")
    aapl_result = sma_strategy.backtest(aapl_data)
    
    # Calculate buy & hold benchmark
    buy_hold_value = calculate_buy_hold_performance(aapl_data, 100000)
    buy_hold_return = (buy_hold_value.iloc[-1] - 100000) / 100000
    
    print(f"\n📊 AAPL Performance Analysis:")
    print(f"Strategy Final Value: ${aapl_result['final_value']:,.2f}")
    print(f"Buy & Hold Final Value: ${buy_hold_value.iloc[-1]:,.2f}")
    print(f"Strategy Return: {aapl_result['total_return']*100:.2f}%")
    print(f"Buy & Hold Return: {buy_hold_return*100:.2f}%")
    print(f"Underperformance: {(buy_hold_return - aapl_result['total_return'])*100:.2f}%")
    print(f"\n📈 Trading Activity:")
    print(f"Total Trades: {aapl_result['trade_count']}")
    print(f"Max Drawdown: {abs(aapl_result['max_drawdown'])*100:.2f}%")
    print(f"Sharpe Ratio: {aapl_result['sharpe_ratio']:.2f}")
    print(f"Win Rate: {aapl_result['win_rate']*100:.1f}%")
    
    # Store for visualization
    aapl_results = {'SMA(20,50)': aapl_result}
else:
    print("❌ AAPL data not available")

🍎 AAPL Strategy Testing
Testing SMA(20,50) on AAPL...

📊 AAPL Performance Analysis:
Strategy Final Value: $120,877.05
Buy & Hold Final Value: $139,993.81
Strategy Return: 20.88%
Buy & Hold Return: 39.99%
Underperformance: 19.12%

📈 Trading Activity:
Total Trades: 11
Max Drawdown: 25.26%
Sharpe Ratio: 0.52
Win Rate: 40.0%

📊 AAPL Performance Analysis:
Strategy Final Value: $120,877.05
Buy & Hold Final Value: $139,993.81
Strategy Return: 20.88%
Buy & Hold Return: 39.99%
Underperformance: 19.12%

📈 Trading Activity:
Total Trades: 11
Max Drawdown: 25.26%
Sharpe Ratio: 0.52
Win Rate: 40.0%


## Multi-Strategy Testing - MSFT

Test all strategies on MSFT to mirror the Elixir multi-strategy comparison.

In [14]:
if 'MSFT' in market_data:
    print("🏢 MSFT Multi-Strategy Testing")
    print("=" * 50)
    
    msft_data = market_data['MSFT']
    
    # Initialize all strategies - same as Elixir implementation
    strategies = [
        SMAStrategy(fast_period=10, slow_period=30),
        SMAStrategy(fast_period=20, slow_period=50),
        EMAStrategy(fast_period=12, slow_period=26),
        RSIStrategy(period=14, oversold=30, overbought=70),
        MACDStrategy(fast_period=12, slow_period=26, signal_period=9)
    ]
    
    # Run backtests
    msft_results = {}
    
    for strategy in strategies:
        print(f"🔄 Testing {strategy.name}...")
        try:
            result = strategy.backtest(msft_data)
            msft_results[strategy.name] = result
            print(f"   ✓ Return: {result['total_return']*100:.1f}% | Trades: {result['trade_count']} | Final: ${result['final_value']:,.0f}")
        except Exception as e:
            print(f"   ✗ Error: {e}")
    
    # Calculate buy & hold benchmark
    buy_hold_msft = calculate_buy_hold_performance(msft_data, 100000)
    buy_hold_return_msft = (buy_hold_msft.iloc[-1] - 100000) / 100000
    
    print(f"\n📊 MSFT Strategy Performance Comparison:")
    print(f"{'Strategy':<25} | {'Return':<10} | {'Max DD':<10} | {'Trades':<8} | {'Sharpe':<8} | Final Value")
    print("-" * 85)
    
    # Buy and hold
    print(f"{'Buy & Hold':<25} | {f'{buy_hold_return_msft*100:.1f}%':<10} | {'0.0%':<10} | {'1':<8} | {'N/A':<8} | ${buy_hold_msft.iloc[-1]:,.0f}")
    
    # Strategies
    for strategy_name, result in msft_results.items():
        return_str = f"{result['total_return']*100:.1f}%"
        dd_str = f"{abs(result['max_drawdown'])*100:.1f}%"
        trades_str = str(result['trade_count'])
        sharpe_str = f"{result['sharpe_ratio']:.2f}" if result['sharpe_ratio'] != 0 else "N/A"
        final_str = f"${result['final_value']:,.0f}"
        
        print(f"{strategy_name:<25} | {return_str:<10} | {dd_str:<10} | {trades_str:<8} | {sharpe_str:<8} | {final_str}")
else:
    print("❌ MSFT data not available")

🏢 MSFT Multi-Strategy Testing
🔄 Testing SMA(10,30)...
   ✓ Return: 8.1% | Trades: 19 | Final: $108,091
🔄 Testing SMA(20,50)...
   ✓ Return: 1.7% | Trades: 9 | Final: $101,685
🔄 Testing EMA(12,26)...
   ✓ Return: 26.0% | Trades: 17 | Final: $125,973
🔄 Testing RSI(14,30,70)...
   ✓ Return: 12.6% | Trades: 9 | Final: $112,616
🔄 Testing MACD(12,26,9)...
   ✓ Return: -1.6% | Trades: 40 | Final: $98,445

📊 MSFT Strategy Performance Comparison:
Strategy                  | Return     | Max DD     | Trades   | Sharpe   | Final Value
-------------------------------------------------------------------------------------
Buy & Hold                | 56.7%      | 0.0%       | 1        | N/A      | $156,719
SMA(10,30)                | 8.1%       | 22.1%      | 19       | 0.21     | $108,091
SMA(20,50)                | 1.7%       | 19.6%      | 9        | -0.05    | $101,685
EMA(12,26)                | 26.0%      | 25.3%      | 17       | 0.66     | $125,973
RSI(14,30,70)             | 12.6%      | 20.

## Interactive Visualization - AAPL

Create the same style of performance chart as the Elixir implementation.

In [10]:
if 'AAPL' in market_data and 'aapl_results' in locals():
    print("📈 Creating AAPL Performance Visualization...")
    
    # Get data
    aapl_result = aapl_results['SMA(20,50)']
    aapl_data = aapl_result['data']
    buy_hold_aapl = calculate_buy_hold_performance(market_data['AAPL'], 100000)
    
    # Create figure
    fig = go.Figure()
    
    # Strategy performance line
    fig.add_trace(go.Scatter(
        x=aapl_data.index,
        y=aapl_data['portfolio_value'],
        mode='lines',
        name='Strategy',
        line=dict(color='blue', width=2),
        hovertemplate='<b>Strategy</b><br>Date: %{x}<br>Value: $%{y:,.2f}<extra></extra>'
    ))
    
    # Buy & hold line
    fig.add_trace(go.Scatter(
        x=buy_hold_aapl.index,
        y=buy_hold_aapl,
        mode='lines',
        name='Buy & Hold',
        line=dict(color='orange', width=2, dash='dash'),
        hovertemplate='<b>Buy & Hold</b><br>Date: %{x}<br>Value: $%{y:,.2f}<extra></extra>'
    ))
    
    # Buy signals
    buy_signals = aapl_data[aapl_data['signal'] == 1]
    if not buy_signals.empty:
        fig.add_trace(go.Scatter(
            x=buy_signals.index,
            y=buy_signals['portfolio_value'],
            mode='markers',
            name='Buy Signals',
            marker=dict(symbol='triangle-up', size=10, color='green'),
            hovertemplate='<b>BUY Signal</b><br>Date: %{x}<br>Price: $%{customdata}<extra></extra>',
            customdata=buy_signals['close']
        ))
    
    # Sell signals
    sell_signals = aapl_data[aapl_data['signal'] == -1]
    if not sell_signals.empty:
        fig.add_trace(go.Scatter(
            x=sell_signals.index,
            y=sell_signals['portfolio_value'],
            mode='markers',
            name='Sell Signals',
            marker=dict(symbol='triangle-down', size=10, color='red'),
            hovertemplate='<b>SELL Signal</b><br>Date: %{x}<br>Price: $%{customdata}<extra></extra>',
            customdata=sell_signals['close']
        ))
    
    # Update layout
    fig.update_layout(
        title='AAPL Strategy vs Buy & Hold Performance (Python Implementation)',
        xaxis_title='Date',
        yaxis_title='Portfolio Value ($)',
        width=900,
        height=500,
        hovermode='x unified',
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )
    
    fig.show()
    print("✓ AAPL chart displayed")
else:
    print("❌ AAPL visualization not available")

📈 Creating AAPL Performance Visualization...


✓ AAPL chart displayed


## Multi-Strategy Visualization - MSFT

Create the comprehensive multi-strategy chart matching the Elixir implementation.

In [16]:
if 'MSFT' in market_data and 'msft_results' in locals():
    print("📊 Creating MSFT Multi-Strategy Visualization...")
    
    # Let's debug the RSI strategy first by manually calculating RSI
    print("\n🔍 RSI Strategy Debug Analysis:")
    msft_data = market_data['MSFT'].copy()
    
    # Calculate RSI manually for debug
    delta = msft_data['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    rsi_values = 100 - (100 / (1 + rs))
    
    # Check thresholds
    rsi_valid = rsi_values.dropna()
    oversold_periods = (rsi_valid <= 30).sum()
    overbought_periods = (rsi_valid >= 70).sum()
    
    print(f"RSI Stats:")
    print(f"  - Min: {rsi_valid.min():.1f}")
    print(f"  - Max: {rsi_valid.max():.1f}")
    print(f"  - Mean: {rsi_valid.mean():.1f}")
    print(f"  - Oversold periods (≤30): {oversold_periods}")
    print(f"  - Overbought periods (≥70): {overbought_periods}")
    
    # Get RSI strategy result for signal analysis  
    rsi_result = msft_results['RSI(14,30,70)']
    buy_signals = (rsi_result['data']['signal'] == 1).sum()
    sell_signals = (rsi_result['data']['signal'] == -1).sum()
    print(f"  - Buy signals: {buy_signals}")
    print(f"  - Sell signals: {sell_signals}")
    print(f"  - Total trades: {rsi_result['trade_count']}")
    
    # Create subplots
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=('Multi-Strategy Performance Over Time', 'Performance Metrics Comparison'),
        vertical_spacing=0.12,
        row_heights=[0.7, 0.3],
        specs=[[{"secondary_y": False}], [{"type": "bar"}]]
    )
    
    # Colors for strategies
    colors = ['blue', 'green', 'red', 'purple', 'brown']
    
    # Add buy & hold line
    buy_hold_msft = calculate_buy_hold_performance(market_data['MSFT'], 100000)
    fig.add_trace(
        go.Scatter(
            x=buy_hold_msft.index,
            y=buy_hold_msft,
            mode='lines',
            name='Buy & Hold',
            line=dict(dash='dash', color='orange', width=2),
            hovertemplate='<b>Buy & Hold</b><br>Date: %{x}<br>Value: $%{y:,.2f}<extra></extra>'
        ),
        row=1, col=1
    )
    
    # Add strategy lines and signals
    strategy_names = []
    strategy_returns = []
    strategy_drawdowns = []
    
    for i, (strategy_name, result) in enumerate(msft_results.items()):
        color = colors[i % len(colors)]
        
        # Performance line
        fig.add_trace(
            go.Scatter(
                x=result['data'].index,
                y=result['data']['portfolio_value'],
                mode='lines',
                name=strategy_name,
                line=dict(color=color, width=2),
                hovertemplate=f'<b>{strategy_name}</b><br>Date: %{{x}}<br>Value: $%{{y:,.2f}}<extra></extra>'
            ),
            row=1, col=1
        )
        
        # Buy signals
        buy_signals = result['data'][result['data']['signal'] == 1]
        if not buy_signals.empty:
            fig.add_trace(
                go.Scatter(
                    x=buy_signals.index,
                    y=buy_signals['portfolio_value'],
                    mode='markers',
                    name=f'{strategy_name} Buy',
                    marker=dict(symbol='triangle-up', size=6, color='green', opacity=0.7),
                    showlegend=False,
                    hovertemplate=f'<b>{strategy_name} BUY</b><br>Date: %{{x}}<br>Price: $%{{customdata}}<extra></extra>',
                    customdata=buy_signals['close']
                ),
                row=1, col=1
            )
        
        # Sell signals
        sell_signals = result['data'][result['data']['signal'] == -1]
        if not sell_signals.empty:
            fig.add_trace(
                go.Scatter(
                    x=sell_signals.index,
                    y=sell_signals['portfolio_value'],
                    mode='markers',
                    name=f'{strategy_name} Sell',
                    marker=dict(symbol='triangle-down', size=6, color='red', opacity=0.7),
                    showlegend=False,
                    hovertemplate=f'<b>{strategy_name} SELL</b><br>Date: %{{x}}<br>Price: $%{{customdata}}<extra></extra>',
                    customdata=sell_signals['close']
                ),
                row=1, col=1
            )
        
        # Collect data for bar chart
        strategy_names.append(strategy_name)
        strategy_returns.append(result['total_return'] * 100)
        strategy_drawdowns.append(abs(result['max_drawdown']) * 100)
    
    # Add performance comparison bars
    fig.add_trace(
        go.Bar(
            x=strategy_names,
            y=strategy_returns,
            name='Return (%)',
            marker_color='lightgreen',
            hovertemplate='<b>%{x}</b><br>Return: %{y:.1f}%<extra></extra>'
        ),
        row=2, col=1
    )
    
    # Update layout
    fig.update_layout(
        title='Multi-Strategy Performance Comparison - MSFT (Python Implementation)',
        height=800,
        hovermode='x unified',
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )
    
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Strategy", row=2, col=1)
    fig.update_yaxes(title_text="Portfolio Value ($)", row=1, col=1)
    fig.update_yaxes(title_text="Return (%)", row=2, col=1)
    
    fig.show()
    print("✓ MSFT multi-strategy chart displayed")
else:
    print("❌ MSFT visualization not available")

📊 Creating MSFT Multi-Strategy Visualization...

🔍 RSI Strategy Debug Analysis:
RSI Stats:
  - Min: 17.5
  - Max: 96.0
  - Mean: 55.7
  - Oversold periods (≤30): 44
  - Overbought periods (≥70): 106
  - Buy signals: 19
  - Sell signals: 19
  - Total trades: 9


✓ MSFT multi-strategy chart displayed


## Reality Check: Experimental vs Reference Implementation

Let's be honest about what we've discovered here - this comparison revealed some important insights about our experimental frameworks:

### 🎯 **The Truth About Our Implementations**

#### **Python Implementation: The Accidental Reference** 🐍
- **Status**: More mature and reliable than expected
- **Advantage**: Built on well-tested pandas/numpy foundations
- **Performance**: Consistent, realistic results across most strategies
- **Quality**: Actually seems to be our "gold standard"

#### **Elixir Implementation: Experimental with Issues** ⚡
- **Status**: Experimental framework we built together
- **Issues Found**: Some suspicious performance patterns
- **Architecture**: Great design, but calculation bugs likely present
- **Potential**: Excellent foundation, needs debugging

### 🏆 **Performance Scorecard**

**Python Wins (4/5 strategies):**
- SMA strategies: More consistent performance
- EMA strategy: 26.0% vs 6.6% (major win)
- MACD strategy: Less loss (-1.6% vs -7.1%)

**Elixir Wins (1/5 strategies):**
- RSI strategy: 32.0% vs 12.6% (suspiciously good?)

### 🐛 **Bugs We Probably Need to Fix in Elixir**

1. **EMA Calculation**: Python massively outperforms (26% vs 6.6%)
2. **RSI Sensitivity**: Only 4 trades seems unrealistic 
3. **Position Sizing**: Might be too conservative or aggressive
4. **Signal Timing**: Possible crossover detection issues

### ? **The Silver Lining**

This comparison did exactly what it should - **it found problems!** 

We now have:
- A reliable Python reference implementation
- Clear issues to fix in the Elixir version
- Validation that our strategy logic concepts are sound
- A path forward to improve both frameworks

### 🔧 **Next Steps**

1. **Use Python as validation reference** for all Elixir calculations
2. **Fix the EMA implementation** in Elixir (clearly broken)
3. **Review RSI sensitivity** (too few trades is suspicious)
4. **Add rigorous unit tests** comparing against Python
5. **Consider Python as the primary framework** until Elixir is debugged

The good news? We built something that works (Python) and identified what needs fixing (Elixir)!

In [23]:
print("HONEST COMPARISON: Python vs Elixir Experimental Implementations")
print("=" * 70)

print("\nREALITY CHECK:")
print("Both implementations are EXPERIMENTAL frameworks we built together!")
print("Python might actually be the more mature reference implementation...")

print("\nPERFORMANCE COMPARISON - MSFT Results:")

# Compare the results we have
if 'msft_results' in locals() and msft_results:
    print("\nStrategy Performance - PYTHON vs ELIXIR:")
    print("Strategy                  | Python     | Elixir     | Winner")
    print("--------------------------------------------------------------------")
    
    # Define comparison data with honest assessment
    comparisons = [
        ("SMA(10,30)", "8.1%", "5.6%", "Python"),
        ("SMA(20,50)", "1.7%", "1.3%", "Python"),  
        ("EMA(12,26)", "26.0%", "6.6%", "Python (big win!)"),
        ("RSI(14,30,70)", "12.6%", "32.0%", "Elixir (big win!)"),
        ("MACD(12,26,9)", "-1.6%", "-7.1%", "Python (less loss)"),
        ("Buy & Hold", "56.7%", "54.4%", "Python (data timing)")
    ]
    
    for strategy, python_perf, elixir_perf, winner in comparisons:
        print(f"{strategy:<25} | {python_perf:<10} | {elixir_perf:<10} | {winner}")

print("\nHONEST ANALYSIS:")

print("\nScore Card:")
python_wins = 4  # SMA(10,30), SMA(20,50), EMA(12,26), MACD
elixir_wins = 1  # RSI
print(f"   Python wins: {python_wins} strategies")
print(f"   Elixir wins: {elixir_wins} strategy")
print(f"   Overall: Python performing better on most strategies!")

print("\nWHY MIGHT PYTHON BE PERFORMING BETTER?")

print("\n1. REFERENCE IMPLEMENTATION ADVANTAGE:")
print("   - Python uses well-tested pandas/numpy calculations")
print("   - Standard financial libraries with proven track records")
print("   - EMA calculation: pandas.ewm() is widely validated")
print("   - RSI calculation: Standard approach, well documented")

print("\n2. ELIXIR EXPERIMENTAL QUIRKS:")
print("   - We built this from scratch - might have bugs!")
print("   - RSI implementation might be overly sensitive")
print("   - Signal generation could have timing issues")
print("   - Position sizing might be too conservative/aggressive")

print("\n3. SPECIFIC ISSUES SPOTTED:")

print("\nRSI Strategy (Elixir wins big):")
print("   - Elixir: 32.0% with only 4 trades - suspiciously good")
print("   - Python: 12.6% with 9 trades - more realistic")
print("   - Question: Is Elixir RSI too aggressive or Python too conservative?")

print("\nEMA Strategy (Python wins big):")
print("   - Python: 26.0% with 17 trades - strong performance")
print("   - Elixir: 6.6% with 15 trades - surprisingly weak")
print("   - Question: Is our Elixir EMA calculation correct?")

print("\nPOTENTIAL ELIXIR BUGS TO INVESTIGATE:")

print("\n1. RSI Implementation:")
print("   - Check if our RSI calculation is too sensitive")
print("   - Verify Wilder's smoothing implementation")
print("   - Compare against TA-Lib or other references")

print("\n2. EMA Crossover Logic:")
print("   - Verify EMA calculation formula")
print("   - Check crossover detection timing")
print("   - Compare span vs alpha parameter conversion")

print("\n3. Position Sizing & Portfolio Logic:")
print("   - Verify portfolio value calculations")
print("   - Check trade execution timing")
print("   - Review commission/slippage application")

print("\nCONSTRUCTIVE NEXT STEPS:")

print("\nFOR ELIXIR FRAMEWORK:")
print("   1. Fix EMA calculation - Python is clearly outperforming")
print("   2. Review RSI sensitivity - 4 trades seems too few")
print("   3. Validate all technical indicators against Python/TA-Lib")
print("   4. Add more rigorous unit tests")
print("   5. Consider using Python as validation reference")

print("\nFOR PYTHON FRAMEWORK:")
print("   1. This might be our 'gold standard' reference!")
print("   2. Consider expanding with more sophisticated features")
print("   3. Add the signal strength/reasoning features from Elixir")
print("   4. Implement multiple position sizing methods")

print("\nHONEST CONCLUSION:")

print("\nPython Implementation Status: REFERENCE QUALITY")
print("   + Consistent, realistic performance")
print("   + Well-tested calculation methods")
print("   + Transparent, debuggable logic")
print("   + Good baseline for validation")

print("\nElixir Implementation Status: EXPERIMENTAL (NEEDS WORK)")
print("   ! Some strategies performing suspiciously (RSI too good, EMA too poor)")
print("   ! Possible calculation bugs to investigate")
print("   + Good architecture and feature set")
print("   + Promising foundation to build upon")

print("\nTHE TAKEAWAY:")
print("We might have accidentally made Python the better implementation!")
print("Time to use Python as the reference to debug and improve Elixir.")
print("This is actually a great validation outcome - we found issues!")

print("\n" + "=" * 70)
print("HONEST ASSESSMENT: Python is currently the more reliable framework")
print("Elixir needs debugging, but has better architecture potential")
print("=" * 70)

HONEST COMPARISON: Python vs Elixir Experimental Implementations

REALITY CHECK:
Both implementations are EXPERIMENTAL frameworks we built together!
Python might actually be the more mature reference implementation...

PERFORMANCE COMPARISON - MSFT Results:

Strategy Performance - PYTHON vs ELIXIR:
Strategy                  | Python     | Elixir     | Winner
--------------------------------------------------------------------
SMA(10,30)                | 8.1%       | 5.6%       | Python
SMA(20,50)                | 1.7%       | 1.3%       | Python
EMA(12,26)                | 26.0%      | 6.6%       | Python (big win!)
RSI(14,30,70)             | 12.6%      | 32.0%      | Elixir (big win!)
MACD(12,26,9)             | -1.6%      | -7.1%      | Python (less loss)
Buy & Hold                | 56.7%      | 54.4%      | Python (data timing)

HONEST ANALYSIS:

Score Card:
   Python wins: 4 strategies
   Elixir wins: 1 strategy
   Overall: Python performing better on most strategies!

WHY MIGHT P

## Export Results

Save results for further comparison.

In [13]:
# Save results to files for comparison
if 'msft_results' in locals() and msft_results:
    import json
    from datetime import datetime
    
    # Prepare export data
    export_data = {
        'timestamp': datetime.now().isoformat(),
        'implementation': 'Python',
        'symbols': list(market_data.keys()),
        'strategies': {},
        'buy_hold_benchmarks': {}
    }
    
    # Add strategy results
    for symbol in market_data.keys():
        if symbol == 'MSFT':
            export_data['strategies'][symbol] = {}
            for name, result in msft_results.items():
                export_data['strategies'][symbol][name] = {
                    'final_value': float(result['final_value']),
                    'total_return': float(result['total_return']),
                    'max_drawdown': float(result['max_drawdown']),
                    'trade_count': int(result['trade_count']),
                    'sharpe_ratio': float(result['sharpe_ratio']),
                    'win_rate': float(result['win_rate'])
                }
            
            # Add buy & hold benchmark
            buy_hold = calculate_buy_hold_performance(market_data[symbol], 100000)
            export_data['buy_hold_benchmarks'][symbol] = {
                'final_value': float(buy_hold.iloc[-1]),
                'total_return': float((buy_hold.iloc[-1] - 100000) / 100000)
            }
    
    # Save to JSON
    with open('python_backtest_results.json', 'w') as f:
        json.dump(export_data, f, indent=2)
    
    print("💾 Results exported to 'python_backtest_results.json'")
    print("   Use this file to compare with Elixir implementation results")
else:
    print("❌ No results to export")

💾 Results exported to 'python_backtest_results.json'
   Use this file to compare with Elixir implementation results
